# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 20 2022 10:44AM,242310,15,8294781,"Brookfield Pharmaceuticals, LLC",Released
1,Jul 20 2022 10:44AM,242310,15,8294782,"Brookfield Pharmaceuticals, LLC",Released
2,Jul 20 2022 10:44AM,242310,15,8294783,"Brookfield Pharmaceuticals, LLC",Released
3,Jul 20 2022 10:44AM,242310,15,8294784,"Brookfield Pharmaceuticals, LLC",Released
4,Jul 20 2022 10:44AM,242310,15,8294785,"Brookfield Pharmaceuticals, LLC",Released
5,Jul 20 2022 10:44AM,242310,15,8294786,"Brookfield Pharmaceuticals, LLC",Released
6,Jul 20 2022 10:44AM,242310,15,8294787,"Brookfield Pharmaceuticals, LLC",Released
7,Jul 20 2022 10:44AM,242310,15,8294788,"Brookfield Pharmaceuticals, LLC",Released
8,Jul 20 2022 10:44AM,242310,15,8294789,"Brookfield Pharmaceuticals, LLC",Released
9,Jul 20 2022 10:44AM,242310,15,8294790,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,242306,Released,2
31,242307,Released,1
32,242308,Released,1
33,242309,Released,16
34,242310,Released,25


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242306,NaN,NaN,2.0
242307,NaN,NaN,1.0
242308,NaN,NaN,1.0
242309,NaN,NaN,16.0
242310,NaN,NaN,25.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242207,0.0,1.0,0.0
242224,0.0,0.0,1.0
242229,9.0,14.0,2.0
242231,0.0,0.0,1.0
242233,0.0,0.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242207,0,1,0
242224,0,0,1
242229,9,14,2
242231,0,0,1
242233,0,0,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242207,0,1,0
1,242224,0,0,1
2,242229,9,14,2
3,242231,0,0,1
4,242233,0,0,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242207,,1,
1,242224,,,1
2,242229,9,14,2
3,242231,,,1
4,242233,,,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 20 2022 10:44AM,242310,15,"Brookfield Pharmaceuticals, LLC"
25,Jul 20 2022 10:43AM,242309,15,"Alliance Pharma, Inc."
41,Jul 20 2022 10:40AM,242308,10,"Senseonics, Incorporated"
42,Jul 20 2022 10:35AM,242307,19,"AdvaGen Pharma, Ltd"
43,Jul 20 2022 10:28AM,242306,10,"Citieffe, Inc."
45,Jul 20 2022 10:26AM,242305,10,"Citieffe, Inc."
46,Jul 20 2022 10:20AM,242304,19,"GUSA Granules USA, Inc."
47,Jul 20 2022 10:06AM,242302,20,Alumier Labs Inc.
60,Jul 20 2022 9:21AM,242300,10,"Nextsource Biotechnology, LLC"
61,Jul 20 2022 9:05AM,242299,19,"Quagen Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 20 2022 10:44AM,242310,15,"Brookfield Pharmaceuticals, LLC",,,25
1,Jul 20 2022 10:43AM,242309,15,"Alliance Pharma, Inc.",,,16
2,Jul 20 2022 10:40AM,242308,10,"Senseonics, Incorporated",,,1
3,Jul 20 2022 10:35AM,242307,19,"AdvaGen Pharma, Ltd",,,1
4,Jul 20 2022 10:28AM,242306,10,"Citieffe, Inc.",,,2
5,Jul 20 2022 10:26AM,242305,10,"Citieffe, Inc.",,,1
6,Jul 20 2022 10:20AM,242304,19,"GUSA Granules USA, Inc.",,,1
7,Jul 20 2022 10:06AM,242302,20,Alumier Labs Inc.,,,13
8,Jul 20 2022 9:21AM,242300,10,"Nextsource Biotechnology, LLC",,,1
9,Jul 20 2022 9:05AM,242299,19,"Quagen Pharmaceuticals, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 10:44AM,242310,15,"Brookfield Pharmaceuticals, LLC",25,,
1,Jul 20 2022 10:43AM,242309,15,"Alliance Pharma, Inc.",16,,
2,Jul 20 2022 10:40AM,242308,10,"Senseonics, Incorporated",1,,
3,Jul 20 2022 10:35AM,242307,19,"AdvaGen Pharma, Ltd",1,,
4,Jul 20 2022 10:28AM,242306,10,"Citieffe, Inc.",2,,
5,Jul 20 2022 10:26AM,242305,10,"Citieffe, Inc.",1,,
6,Jul 20 2022 10:20AM,242304,19,"GUSA Granules USA, Inc.",1,,
7,Jul 20 2022 10:06AM,242302,20,Alumier Labs Inc.,13,,
8,Jul 20 2022 9:21AM,242300,10,"Nextsource Biotechnology, LLC",1,,
9,Jul 20 2022 9:05AM,242299,19,"Quagen Pharmaceuticals, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 10:44AM,242310,15,"Brookfield Pharmaceuticals, LLC",25,,
1,Jul 20 2022 10:43AM,242309,15,"Alliance Pharma, Inc.",16,,
2,Jul 20 2022 10:40AM,242308,10,"Senseonics, Incorporated",1,,
3,Jul 20 2022 10:35AM,242307,19,"AdvaGen Pharma, Ltd",1,,
4,Jul 20 2022 10:28AM,242306,10,"Citieffe, Inc.",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 10:44AM,242310,15,"Brookfield Pharmaceuticals, LLC",25.0,NaN,NaN
1,Jul 20 2022 10:43AM,242309,15,"Alliance Pharma, Inc.",16.0,NaN,NaN
2,Jul 20 2022 10:40AM,242308,10,"Senseonics, Incorporated",1.0,NaN,NaN
3,Jul 20 2022 10:35AM,242307,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
4,Jul 20 2022 10:28AM,242306,10,"Citieffe, Inc.",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 10:44AM,242310,15,"Brookfield Pharmaceuticals, LLC",25.0,0.0,0.0
1,Jul 20 2022 10:43AM,242309,15,"Alliance Pharma, Inc.",16.0,0.0,0.0
2,Jul 20 2022 10:40AM,242308,10,"Senseonics, Incorporated",1.0,0.0,0.0
3,Jul 20 2022 10:35AM,242307,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
4,Jul 20 2022 10:28AM,242306,10,"Citieffe, Inc.",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2907361,124.0,49.0,0.0
15,484619,41.0,0.0,0.0
16,969164,4.0,0.0,0.0
18,484493,1.0,1.0,0.0
19,1938160,8.0,17.0,13.0
20,484535,32.0,0.0,0.0
22,242275,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2907361,124.0,49.0,0.0
1,15,484619,41.0,0.0,0.0
2,16,969164,4.0,0.0,0.0
3,18,484493,1.0,1.0,0.0
4,19,1938160,8.0,17.0,13.0
5,20,484535,32.0,0.0,0.0
6,22,242275,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,124.0,49.0,0.0
1,15,41.0,0.0,0.0
2,16,4.0,0.0,0.0
3,18,1.0,1.0,0.0
4,19,8.0,17.0,13.0
5,20,32.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,124.0
1,15,Released,41.0
2,16,Released,4.0
3,18,Released,1.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,13.0,0.0,0.0
Executing,49.0,0.0,0.0,1.0,17.0,0.0,1.0
Released,124.0,41.0,4.0,1.0,8.0,32.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,13.0,0.0,0.0
1,Executing,49.0,0.0,0.0,1.0,17.0,0.0,1.0
2,Released,124.0,41.0,4.0,1.0,8.0,32.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,13.0,0.0,0.0
1,Executing,49.0,0.0,0.0,1.0,17.0,0.0,1.0
2,Released,124.0,41.0,4.0,1.0,8.0,32.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()